In [2]:
#!/usr/bin/env python3
import sys
import smach
import rospy
import cv2 as cv
import numpy as np
from std_srvs.srv import Empty
import moveit_commander
import moveit_msgs.msg
import tf2_ros as tf2
from tf2_sensor_msgs.tf2_sensor_msgs import do_transform_cloud
from utils_takeshi import *
from grasp_utils import *

In [3]:
    global head, whole_body, arm, tfbuff, lis, broad, tf_static_broad, gaze
    global rgbd, hand_cam, wrist, gripper, grasp_base, clear_octo_client, service_client, AR_starter, AR_stopper

    moveit_commander.roscpp_initialize(sys.argv)
    rospy.init_node('Pruebas_de_graspeo_v2')
    head = moveit_commander.MoveGroupCommander('head')
    whole_body = moveit_commander.MoveGroupCommander('whole_body_light')
    arm =  moveit_commander.MoveGroupCommander('arm')
    
    tfbuff = tf2.Buffer()
    lis = tf2.TransformListener(tfbuff)
    
    broad = tf2.TransformBroadcaster()
    whole_body.set_workspace([-6.0, -6.0, 6.0, 6.0]) 
    
    tf_man = TF_MANAGER()
    rgbd = RGBD()
    hand_cam = HAND_RGB()
    wrist = WRIST_SENSOR()
    gripper = GRIPPER()
    grasp_base = OMNIBASE()
    gaze = GAZE()

    clear_octo_client = rospy.ServiceProxy('/clear_octomap', Empty)
    AR_starter = rospy.ServiceProxy('/marker/start_recognition',Empty)
    AR_stopper = rospy.ServiceProxy('/marker/stop_recognition',Empty)
    
    head.set_planning_time(0.3)
    head.set_num_planning_attempts(1)

[ WARN] [1666368910.833226937, 2374.632000000]: Link hand_l_finger_vacuum_frame has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1666368910.843243965, 2374.632000000]: Link head_l_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1666368910.843360062, 2374.632000000]: Link head_r_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1666368910.844783552, 2374.632000000]: Group state 'neutral' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.
[ WARN] [1666368910.844861020, 2374.632000000]: Group state 'go' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.

[ WARN] [1666369109.747955325, 2398.591000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2398.643000 according to authority unknown_publisher
[ WARN] [1666369109.748042967, 2398.591000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2398.643000 according to authority /pose_integrator
[ WARN] [1666369109.748117587, 2398.591000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2398.643000 according to authority /pose_integrator
[ WARN] [1666369109.748169295, 2398.591000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2398.643000 according to authority /pose_integrator
[ WARN] [1666369112.755575120, 2398.986000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2399.037000 according to authority /pose_integrator
[ WARN] [1666369112.755671300, 2398.986000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

[ WARN] [1666369207.509267955, 2411.450000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2411.473000 according to authority unknown_publisher
[ WARN] [1666369207.510292124, 2411.450000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2411.473000 according to authority /pose_integrator
[ WARN] [1666369207.510672332, 2411.450000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2411.473000 according to authority /pose_integrator
[ WARN] [1666369207.510991426, 2411.450000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2411.473000 according to authority /pose_integrator
[ WARN] [1666369217.605208399, 2413.089000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2413.144000 according to authority /pose_integrator
[ WARN] [1666369217.605321304, 2413.089000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

[ WARN] [1666369340.058431292, 2433.099000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2433.119000 according to authority /pose_integrator
[ WARN] [1666369340.058560035, 2433.099000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2433.119000 according to authority /pose_integrator
[ WARN] [1666369340.058641117, 2433.099000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2433.119000 according to authority /pose_integrator
[ WARN] [1666369340.061642912, 2433.099000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2433.119000 according to authority unknown_publisher
[ WARN] [1666369360.278778838, 2435.470000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2435.522000 according to authority /pose_integrator
[ WARN] [1666369360.278901294, 2435.470000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

[ WARN] [1666369485.696163252, 2454.878000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2454.926000 according to authority /pose_integrator
[ WARN] [1666369485.696277924, 2454.878000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2454.926000 according to authority /pose_integrator
[ WARN] [1666369485.696361183, 2454.878000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2454.926000 according to authority /pose_integrator
[ WARN] [1666369485.696586501, 2454.878000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2454.926000 according to authority unknown_publisher
[ WARN] [1666369486.379243056, 2455.039000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2455.039000 according to authority /pose_integrator
[ WARN] [1666369486.379357123, 2455.039000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

...

[ WARN] [1666369869.552081959, 2518.398000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2518.466000 according to authority /pose_integrator
[ WARN] [1666369869.552171902, 2518.398000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2518.466000 according to authority /pose_integrator
[ WARN] [1666369869.552222282, 2518.398000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2518.466000 according to authority /pose_integrator
[ WARN] [1666369869.553340081, 2518.398000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2518.466000 according to authority unknown_publisher
[ WARN] [1666369871.519723288, 2518.850000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2518.881000 according to authority /pose_integrator
[ WARN] [1666369871.519817136, 2518.850000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

[ WARN] [1666370071.538527026, 2555.165000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2555.196000 according to authority unknown_publisher
[ WARN] [1666370071.539081783, 2555.165000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2555.196000 according to authority /pose_integrator
[ WARN] [1666370071.539262609, 2555.165000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2555.196000 according to authority /pose_integrator
[ WARN] [1666370071.539426612, 2555.165000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2555.196000 according to authority /pose_integrator
[ WARN] [1666370094.907322372, 2559.356000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2559.384000 according to authority /pose_integrator
[ WARN] [1666370094.907436663, 2559.356000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

[ WARN] [1666370296.939630066, 2596.726000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2596.758000 according to authority /pose_integrator
[ WARN] [1666370296.939764933, 2596.726000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2596.758000 according to authority /pose_integrator
[ WARN] [1666370296.939837686, 2596.726000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2596.758000 according to authority /pose_integrator
[ WARN] [1666370296.940285917, 2596.726000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2596.758000 according to authority unknown_publisher
[ WARN] [1666370309.402018812, 2599.055000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2599.087000 according to authority /pose_integrator
[ WARN] [1666370309.402107435, 2599.055000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

[ WARN] [1666370401.866636657, 2616.248000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2616.307000 according to authority unknown_publisher
[ WARN] [1666370401.868522922, 2616.248000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2616.307000 according to authority /pose_integrator
[ WARN] [1666370401.868625678, 2616.248000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2616.307000 according to authority /pose_integrator
[ WARN] [1666370401.868704349, 2616.248000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2616.307000 according to authority /pose_integrator
[ WARN] [1666370435.345029320, 2622.847000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2622.898000 according to authority /pose_integrator
[ WARN] [1666370435.345134140, 2622.847000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

[ WARN] [1666370654.209502811, 2664.114000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2664.134000 according to authority unknown_publisher
[ WARN] [1666370654.209706738, 2664.114000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2664.134000 according to authority /pose_integrator
[ WARN] [1666370654.209783585, 2664.114000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2664.134000 according to authority /pose_integrator
[ WARN] [1666370654.209835224, 2664.114000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2664.134000 according to authority /pose_integrator


In [4]:
        # State initial
        try:
            clear_octo_client()
        except:
            print('cant clear octomap')
        AR_stopper.call()
        #Takeshi neutral
        arm.set_named_target('go')
        arm.go()
        gripper.steady()
        head.set_named_target('neutral')
        succ = head.go()
#         if succ:
#             return 'succ'
#         else:
#             return 'failed'

[ WARN] [1666370667.335323855, 2667.010000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2667.034000 according to authority /pose_integrator
[ WARN] [1666370667.335405009, 2667.010000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2667.034000 according to authority /pose_integrator
[ WARN] [1666370667.335450263, 2667.010000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2667.034000 according to authority /pose_integrator
[ WARN] [1666370667.335843064, 2667.010000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2667.034000 according to authority unknown_publisher
[ WARN] [1666370672.373307841, 2668.071000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2668.103000 according to authority unknown_publisher
[ WARN] [1666370672.374317788, 2668.071000000]: TF_REPEATED_DATA ignoring data with redundant timestamp

[ WARN] [1666370739.826113965, 2680.308000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2680.308000 according to authority /pose_integrator
[ WARN] [1666370739.826225005, 2680.308000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2680.308000 according to authority /pose_integrator
[ WARN] [1666370739.826275438, 2680.308000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2680.308000 according to authority /pose_integrator
[ WARN] [1666370739.833835603, 2680.308000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2680.308000 according to authority unknown_publisher
[ WARN] [1666370762.123771848, 2683.956000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2684.045000 according to authority /pose_integrator
[ WARN] [1666370762.123903794, 2683.956000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

[ WARN] [1666370900.992876448, 2706.501000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2706.538000 according to authority /pose_integrator
[ WARN] [1666370900.993019975, 2706.501000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2706.538000 according to authority /pose_integrator
[ WARN] [1666370900.993147390, 2706.501000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2706.538000 according to authority /pose_integrator
[ WARN] [1666370900.997368668, 2706.501000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2706.538000 according to authority unknown_publisher
[ WARN] [1666370909.326124523, 2707.714000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2707.763000 according to authority unknown_publisher
[ WARN] [1666370909.345679921, 2707.763000000]: TF_REPEATED_DATA ignoring data with redundant timestamp

[ WARN] [1666371009.545769288, 2724.548000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2724.591000 according to authority unknown_publisher
[ WARN] [1666371009.549636165, 2724.548000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2724.591000 according to authority /pose_integrator
[ WARN] [1666371009.549739486, 2724.548000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2724.591000 according to authority /pose_integrator
[ WARN] [1666371009.549819632, 2724.548000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2724.591000 according to authority /pose_integrator
[ WARN] [1666371022.917051393, 2726.772000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2726.772000 according to authority unknown_publisher
[ WARN] [1666371022.921552202, 2726.772000000]: TF_REPEATED_DATA ignoring data with redundant timestamp

[ WARN] [1666371138.623733299, 2745.242000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2745.270000 according to authority unknown_publisher
[ WARN] [1666371138.626233681, 2745.242000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2745.270000 according to authority /pose_integrator
[ WARN] [1666371138.626313293, 2745.242000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2745.270000 according to authority /pose_integrator
[ WARN] [1666371138.626382085, 2745.242000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2745.270000 according to authority /pose_integrator
[ WARN] [1666371178.415533378, 2751.580000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2751.613000 according to authority /pose_integrator
[ WARN] [1666371178.415630851, 2751.580000000]: TF_REPEATED_DATA ignoring data with redundant timestamp 

In [6]:
        # State Find AR marker
        try:
            AR_starter.call()
            clear_octo_client()
        except:
            print('cant clear octomap')
        #Takeshi looks for AR marker
        
        rospy.sleep(0.1)
        hcp = head.get_current_joint_values()
        hcp[0] = 0.5
        hcp[1] = -0.2
        head.set_joint_value_target(hcp)
        head.go()
        succ = False
        flag = True
#         talk("I am going to find any AR marker")
        rospy.sleep(0.3)
        t = tfbuff.lookup_transform('base_link', 'ar_marker/201', rospy.Time(0) )
        rospy.sleep(0.3)
        trans, rot = tf2_obj_2_arr(t)
        publish_point_tf(trans, rot, 'AR', ref = 'map')
        hcp = [0.0,0.0]
        head.set_joint_value_target(hcp)
        head.go()
        while not succ:
            try:
                t = tfbuff.lookup_transform('base_link', 'Point AR', rospy.Time(0) )
                rospy.sleep(0.3)
                trans, rot = tf2_obj_2_arr(t)
                distanceX = trans[0]
                
                print(distanceX)
#                 flag = not flag
#                 if distanceX < 0.60 and distanceX > 0.55 and flag:
#                     hcp = head.get_current_joint_values()
#                     hcp[0] += 0.4
#                     hcp[1] = -0.2
#                     head.set_joint_value_target(hcp)
#                     head.go()
                if distanceX < 0.10:
                    succ = True
                else:
                    grasp_base.tiny_move(velX=0.6,std_time=0.1)
            except:
                grasp_base.tiny_move(velX=0.5,std_time=0.1)
        if succ:
            return 'succ'
        else:
            return 'failed'

0.026026329476792787


SyntaxError: 'return' outside function (2600629359.py, line 48)

In [11]:
        # State AR alignment
        succ = False
        THRESHOLD = 0.09
        hcp = [0.6,-0.1]
        flag = True
        #talk("I am going to align with the table")
        while not succ:
            try:
                t = tfbuff.lookup_transform('base_link','ar_marker/201',rospy.Time(0))
                trans, rot = tf2_obj_2_arr(t)
                euler = tf.transformations.euler_from_quaternion(rot)
                theta = euler[2]
                e = theta + 1.57
                print(e)
                if abs(e) < THRESHOLD:
#                     talk("ready")
                    succ = True
                else:
                    rospy.sleep(0.55)
                    grasp_base.tiny_move(velT = 0.2*e, std_time=0.2)
                    flag = not flag
                    if flag:
                        hcp = gaze.relative(trans[0],trans[1],trans[2])
#                         hcp[0] -= 0.1 * (e/abs(e))
                        print(hcp)
                        head.set_joint_value_target(hcp)
                        head.go()
            except:
                hcp[0] -= 0.2
                if hcp[0] > -1.2:
                    hcp[0] = 0.0
                head.set_joint_value_target(hcp)
                head.go()
                
        
        if succ:
            return 'succ'
        else:
            return 'failed'

0.8860280059131496
0.8934027819807039
0.905347139928809
0.8918680814420555
0.5672625995371723
0.532887902448232
0.4932425607944644
0.4932425607944644
0.4932425607944644
0.4932425607944644
0.4932425607944644
0.4932425607944644
0.4932425607944644
0.4932425607944644
0.4932425607944644
0.4932425607944644
0.4932425607944644
0.4932425607944644


KeyboardInterrupt: 

In [22]:
        # State Pre grasp pose
        succ = False
#         talk("I will reach the cassette")
        gripper.open()
        height = 0
        try:
            t = tfbuff.lookup_transform('base_link','ar_marker/201',rospy.Time(0))
            trans,_ = tf2_obj_2_arr(t)
            height = trans[2]
#         grasp_from_above_joints = [0.59,-1.3376,0,-1.8275,0.0,0.0]
        except:
            print('no pude bebe')
        print(height)
        grasp_from_above_joints = [height - 0.102,-1.3376,0,-1.8275,0.0,0.0]
        arm.set_joint_value_target(grasp_from_above_joints)
        succ = arm.go()
        if succ:
            return 'succ'
        else:
            return 'failed'

0.7769439040538094


SyntaxError: 'return' outside function (2998102668.py, line 18)

In [25]:
        # State AR adjustment

        try:
            clear_octo_client()
        except:
            print('cant clear octomap')
#         scene.remove_world_object()
        #Takeshi gets close to the cassette
        AR_starter.call()
        succ = False
        X_OFFSET = 0.0
        Y_OFFSET = 0.19
        Z_OFFSET = 0.135

        THRESHOLD = 0.025

        hcp = head.get_current_joint_values()
        hcp[0] = -0.1
        hcp[1] = -0.5
        head.set_joint_value_target(hcp)
        head.go()
        succ = False
        while not succ:
            try:
                t = tfbuff.lookup_transform('hand_palm_link', 'ar_marker/201', rospy.Time(0) )
#         t = tfbuff.lookup_transform('hand_palm_link', 'ar_marker/4000', rospy.Time(0) )
                traf = t.transform.translation
                rospy.sleep(.6)
        # tiny_move_base(y = 0.163)
                ex = traf.x + X_OFFSET
                ey = -traf.y + Y_OFFSET
                print(ex, ey)
                if abs(ex) > THRESHOLD:
                    grasp_base.tiny_move(velX = ex, MAX_VEL = 0.05)#, y = -traf.y + Y_OFFSET)
                if abs(ey) > THRESHOLD:
                    grasp_base.tiny_move(velY = ey, MAX_VEL = 0.05)
                if (abs(ex) <= THRESHOLD and abs(ey) <= THRESHOLD):
                    hcp[0] = 0
                    head.set_joint_value_target(hcp)
                    head.go()
                    talk("I am almost there")
                    succ = True
            except:
                hcp = head.get_current_joint_values()
                hcp[0] -= 0.1   
                print(hcp[0])
                head.set_joint_value_target(hcp)
                head.go()
                if hcp[0] < -1:
                    hcp[0] = 0.1
                    head.set_joint_value_target(hcp)
                    head.go()
                    print('Ive lost the reference')
                    succ = False
                    break
        if succ:
            stopper.call()
            return 'succ'
        else:
            return 'failed'

0.059632232431935384 -0.06474673001783732
0.054802662942152525 -0.0662636480933218
0.01577228061981839 -0.013003793896080224
-0.10570690154699616
-0.0016034904384321091 0.017272083415478534
-0.10716390154699598
-0.0016034904384321091 0.017272083415478534
-0.10349790154699604
-0.0016034904384321091 0.017272083415478534
-0.10984790154699589
-0.18958490154699606
-0.287712901546996
-0.004946757895945897 0.03301748338564889
-0.00099578159547975 0.0332640329948459
0.0002785311597117701 0.0036912659273853343
-0.100828901546996
-0.0008844766854152475 -0.03531892350132959
-0.0012008608503484286 -0.03542449167850481
-0.0005164636666894484 -0.017160470313466125
-0.10165690154699583
-0.0016295019424446 0.01588159417771215
-0.10863690154699582
-0.0016295019424446 0.01588159417771215
-0.10136490154699587
-0.19972390154699618
-0.2957209015469958
-0.39317390154699605
-0.007732870228937427 0.051850417221427525
-0.0027885023148632726 0.05257357667208559
-0.0032060706979730735 0.02260229840844155
-0.1029

KeyboardInterrupt: 

In [27]:
        # State color adjustment
        try:
            clear_octo_client()
        except:
            print('cant clear octomap')

        #Takeshi detects the cassette by color and go for it
        succ = False
        THRESHOLD = 15
        goalPos = [258.61,261.75]
        while not succ:
            [currentPos] = hand_cam.color_segmentator(color = 'orange')
#     print(currentPos)
            ex = -(goalPos[0]-currentPos[0]) 
            ey = (goalPos[1]-currentPos[1])
            print(ex, ey)
            if abs(ex) > THRESHOLD:
                grasp_base.tiny_move(velX = ex, std_time=0.1, MAX_VEL=0.01)#, y = -traf.y + Y_OFFSET)
                rospy.sleep(0.5)
            if abs(ey) > THRESHOLD:
                grasp_base.tiny_move(velY = ey, std_time=0.1, MAX_VEL=0.01)
                rospy.sleep(0.5)
            if (abs(ex) <= THRESHOLD and abs(ey) <= THRESHOLD):
#                 talk("done, now i will take it")
                succ = True
        if succ:
            return 'succ'
        else:
            return 'failed'

-73.56843015750067 74.09856700232379
-71.61588154835181 74.84848173984406
-73.28697283311772 74.14573091849934
-71.76152727757773 74.39732559690543
-69.41172054946583 74.68742195088109
-66.8344425490435 74.62157304739856
-53.65478952772074 58.475359342915795
-52.643629270917415 57.449757869249396
-34.54791802435261 42.92732807408677
-28.6322459241201 36.50971354563464
-25.498491547464255 36.269505851755525
-1.9503912079048291 -2.9543758822343307


SyntaxError: 'return' outside function (1067665401.py, line 27)

In [29]:
        # State grasp table
        gripper.open()
        rospy.sleep(0.3)
        acp = arm.get_current_joint_values()
        acp[0] -= 0.03
#         acp = [0.56,-1.3376,0,-1.8275,0.0,0.0]
        arm.set_joint_value_target(acp)
        arm.go()
        gripper.close()
        rospy.sleep(0.3)
        
        check_grasp_joints=[0.69,-1.3376,0,-1.8275,0.0,0.0]
        arm.set_joint_value_target(check_grasp_joints)
        arm.go()
        
        check_grasp_joints=[0.69,-1.3376,-0.8,-1.8275,0.0,0.0]
        arm.set_joint_value_target(check_grasp_joints)
        arm.go()

        THRESHOLD = 30
        goalPos = [233.80,268.74]
        [currentPos] = hand_cam.color_segmentator(color = 'orange')
        ex = -(goalPos[0]-currentPos[0]) 
        ey = (goalPos[1]-currentPos[1])
        if (abs(ex) <= THRESHOLD and abs(ey) <= THRESHOLD):
            talk("I have the cassete")
            return 'succ'
        else:
            talk("Something went wrong, i will try again")
            return 'tries'

SyntaxError: 'return' outside function (1553936197.py, line 27)

In [30]:
        # State post grasp pose
        acp = arm.get_current_joint_values()
        acp[0] = 0.69
        arm.set_joint_value_target(acp)
        arm.go()
        rospy.sleep(0.3)
        grasp_base.tiny_move(velX = -0.6, std_time=0.9)
        arm.set_named_target('go')
        succ = arm.go()
        if succ:
            return 'succ'
        else:
            return 'failed'

SyntaxError: 'return' outside function (760905894.py, line 11)

In [ ]:
# State right shift
        grasp_base.tiny_move(velY = -0.8, std_time=0.8)
        succ = True
        if succ:
            return 'succ'
        else:
            return 'failed'

In [10]:
gripper.steady()

In [9]:
# rospy.init_node('Pruebas_TF')
# tf_man = TF_MANAGER()
rospy.sleep(0.5)
tf_man.pub_static_tf(trans = [1,0,1], pointName = 'marker', ref='head_rgbd_sensor_link')

In [7]:
tf_man.change_ref_frame_tf(tf_name='marker', newFrame='map')

True

In [9]:
gripper = GRIPPER()
gripper.steady()

In [8]:
trans, rot = tf_man.getTF('base_link', 'marker')
# gaze = GAZE()
print(trans)
gaze.relative(trans[0],trans[1],trans[2])

[0.9392966689769481, 1.0212325984889241, 0.972171211690557]


[0.7854678577699192, -5.104639176899594e-07]